# Indicadores Sócio-Econômicos de Escolas Públicas (SIOPE)
---

## 1. Carregando as bibliotecas e configurações para a coleta

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

campos2 = ['MUNICIPIO','UF','ANO', 
          
          'LEGAL_PORC_IMPOSTOS', 
        'LEGAL_PORC_FUNDEB_REMUNERACAO', 
        'LEGAL_PORC_FUNDEB_OUTROS', 
        'LEGAL_PORC_FUNDEB_NAOAPLICADO',
          
        'FIN_PORC_FUNDEB_INFANTIL', 
        'FIN_PORC_FUNDEB_FUNDAMENTAL', 
        'FIN_PORC_DESPESAS_INFANTIL_TOTALEDUCACAO', 
        'FIN_PORC_DESPESAS_FUNDAMENAL_TOTALEDUCACAO', 
        'FIN_PORC_DESPESAS_MEDIO_TOTALEDUCACAO',
        'FIN_PORC_DESPESAS_SUPERIOR_TOTALEDUCACAO',
        'FIN_PORC_DESPESAS_EDUCACAO_TOTALGERAL',
        'FIN_PORC_DESPESAS_ALIMENTACAO_TOTALEDUCACAO',
        'FIN_INVEST_MATERIAL_ALUNO_BASICA',
        'FIN_PORC_DESPESAS_EDUCACAO_TOTALMDE',
        'FIN_INVEST_EDUCACAO_TOTALMDE',
          
        'PESSOAL_PORC_APOSENTADORIA_TOTALMDE', 
        'PESSOAL_PORC_ENCARGOS_TOTALMDE', 
        'PESSOAL_PORC_PROFESSORES_TOTALMDE', 
        'PESSOAL_PORC_PROFISSIONAIS_TOTALMDE',
          
        'ALUNO_INV_INFANTIL', 
        'ALUNO_INV_FUNDAMENTAL', 
        'ALUNO_INV_MEDIO', 
        'ALUNO_INV_SUPERIOR', 
        'ALUNO_INV_EJA', 
        'ALUNO_INV_ESPECIAL', 
        'ALUNO_INV_PROFISSIONAL', 
        'ALUNO_INV_EDBASICA', 
        'ALUNO_INV', 
        'ALUNO_DESPESA_DOCENTE_EDBASICA', 
        'ALUNO_DESPESA_PROFISSIONAL_EDBASICA',
        'ALUNO_PORC_INV_SUPERIOR_EDBASICA',
          
        'DESENVEDUC_IDEB_INICIAIS', 
        'DESENVEDUC_IDEB_FINAIS', 
        'DESENVEDUC_TXAPROV_FUNDAMENTAL', 
        'DESENVEDUC_TXAPROV_MEDIO', 
        'DESENVEDUC_TXREPROV_FUNDAMENTAL', 
        'DESENVEDUC_TXREPROV_MEDIO', 
        'DESENVEDUC_TXRABANDONO_FUNDAMENTAL', 
        'DESENVEDUC_TXRABANDONO_MEDIO',
          
        'COMPRECEITA_PORC_FNDE_TOTAL', 
        'COMPRECEITA_PORC_IMPOSTOS_TOTAL', 
        'COMPRECEITA_PORC_CONSTIT_TOTAL',
          
        'RESULTADO_SITUACAO', 
        'RESULTADO_SALDO_FUNDEB']
campos = ['CIDADE','UF','ANO', 
          
          '1.1', 
        '1.2', 
        '1.3', 
        '1.4',
          
        '2.1', 
        '2.2', 
        '2.3', 
        '2.4', 
        '2.5',
        '2.6',
        '2.7',
        '2.8',
        '2.9',
        '2.10',
        '2.11',
        '2.12',
          
        '3.1', 
        '3.2', 
        '3.3', 
        '3.4', 
        '3.5', 
        '3.6',
          
        '4.1', 
        '4.2', 
        '4.3', 
        '4.4', 
        '4.5',
        '4.6',
        '4.7',
        '4.8',
        '4.9',
        '4.10',
        '4.11',
        '4.12',
          
        '5.1', 
        '5.2', 
        '5.3', 
        '5.4', 
        '5.5',
        '5.6',
        '5.7',
        '5.8',
          
        '6.1', 
        '6.2', 
        '6.3',
          
        '7.1', 
        '7.2']

## 2. Coletando UF's e Municípios da base de dados SIOPE
---


In [2]:
requests.packages.urllib3.disable_warnings()

In [3]:
# Capturando as uf
# Capturando / tratando conteudo HTML do intervalo (Parser)
page = requests.get('https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=ATUALIZAR&anoPaginacao=-1&paginacao=&pag=result&cod_uf=12&municipios=270010', verify=False)
if page.status_code == 200:
    content = BeautifulSoup(page.content, 'html.parser')
    #page.close()

    # Listando UF's
    ufs = []
    lista_uf = content.find('select', {'name':'cod_uf'})
    ufs = pd.DataFrame({'Codigo' : [ uf['value'] for uf in lista_uf.find_all('option') ], 
                        'Nome' : [uf.get_text().replace('\n', '').replace('\t', '') for uf in lista_uf.find_all('option') ]})
    # Listando municipios, por UF
    municipios = pd.DataFrame(columns=['Codigo', 'Nome', 'UF', 'CodUF'])
    for idx, uf in ufs.iterrows():
        # Capturando as uf
        # Capturando / tratando conteudo HTML do intervalo (Parser)
        page = requests.get('https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2014&paginacao=-&pag=result&cod_uf='+uf['Codigo']+'&municipios=355030', verify=False)
        content = BeautifulSoup(page.content, 'html.parser')

        lista_municipios = content.find('select', {'name':'municipios'})
        dados = pd.DataFrame({'Codigo': [ m['value'] for m in lista_municipios.find_all('option') ],
                              'Nome': [ m.get_text().replace('\n', '').replace('\t', '').strip() for m in lista_municipios.find_all('option') ],
                              'UF': uf['Nome'].strip(),
                              'CodUF': uf['Codigo']})
        municipios = municipios.append(dados, ignore_index=True)

    # Criando CSV 
    salvar = input('Deseja salvar os dados? (S ou N): ')
    if salvar.lower() == 's':
        ufs.to_csv('dados_siope/ufs.csv', sep=';', encoding='utf-8', index=False)
        municipios.to_csv('dados_siope/municipios.csv', sep=';', encoding='utf-8', index=False)
else:
    print("Erro "+str(page.status_code)+": Houve um problema com a coleta de dados, tente novamente mais tarde.")

Deseja salvar os dados? (S ou N): s


## 3. Definindo as funções de coleta dos dados
---
De acordo com o template disponível no site do SIOPE e a estrutura do conjunto de dados estabelecido anteriormente, duas funções de raspagem são estabelecidas, as quais visam coletar individualmente dois intervalos de tempo, permitindo analisar a evolução histórica de cada município dentro do período 2008 a 2016.

In [4]:
# 2008 - 2013
def dados2008_2013( cod_municipio, municipio, cod_uf, uf, campos, debug = False ):

    # Indicadores completos Ex.: https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2007&paginacao=0&pag=result&cod_uf=35&municipios=355030
    
    intervalo = list(range(2008, 2014))
    # Capturando / tratando conteudo HTML do intervalo (Parser)
    page = requests.get('https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2014&paginacao=-&pag=result&cod_uf='+ str(cod_uf) +'&municipios='+ str(cod_municipio), verify=False)
    content = BeautifulSoup(page.content, 'html.parser')
    #************ HTML LOCAL *******************
    #page = open('teste-2008-2013.html')
    #content = BeautifulSoup(page, 'html.parser')
    #page.close()
    #************ HTML LOCAL *******************
    
    # Capturando dados dos indicadores, por ano 'a' do intervalo    
    registro = []
    for a in range(0, 6):

        #cidade = 'São Paulo' 
        #uf = 'SP'
        ano = intervalo[a]
        if debug==True:
            print('Ano: ', ano)
        
        # Novo registro Municipio / Ano
        data = {}
        data["MUNICIPIO"] = municipio
        data["UF"] = uf
        data["ANO"] = ano

        # Indicadores (tabelas)
        indicadores = content.find_all('table', class_='table')
        for i in range(0, len(indicadores)):

            #print('Indicador: ', i+1)        

            # Subindicador 'j_i', do indicador 'i'
            subindicador_i = indicadores[i].findAll(True, {"class":["rowA", "rowB"]})
            for j in range(0, len(subindicador_i)):
                
                c = subindicador_i[j].find('div', class_='codigo')
                c = c.get_text().replace('\n', '').replace('\t', '').replace('\t', '').replace('\r', '').replace('\xa0', '')
                
                # Dados do subindicador 'j_i' para o ano 'a'
                valores = subindicador_i[j].find_all('div', class_='number')
                v = valores[a].get_text()
                # Tratamentos
                v = v.replace('\n', '').replace('\t', '').replace('\t', '').replace('\r', '').replace('\xa0', '') # Chars
                v = v.replace('R$', '').replace('%', '') # Symbols
                v = v.replace('.', '').replace(',', '.') # Decimal format

                #print('Subindicador ', i+1, '.', j+1, ': ', v)
                data[c] = v

        #registro = [municipio, uf, str(ano)] + registro
        registro.append(data)
        #registro = []

    # Arquivando registro  
    dados = pd.DataFrame(registro)
    #dados = dados.append(pd.DataFrame(registro), ignore_index=True)
    #dados = dados.append(pd.DataFrame(registro, columns=campos), ignore_index=True)
        
    return dados

In [5]:
# 2014 - 2016
def dados2014_2016( cod_municipio, municipio, cod_uf, uf, campos, debug = False ):
    
    intervalo = list(range(2011, 2017))
    # Capturando / tratando conteudo HTML do intervalo (Parser)
#    page = requests.get('https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2017&paginacao=-&pag=result&cod_uf='+cod_uf+'&municipios='+cod_municipio, verify=False)
    page = requests.get('https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2017&paginacao=-&pag=result&cod_uf='+ str(cod_uf) +'&municipios='+ str(cod_municipio), verify=False)
    content = BeautifulSoup(page.content, 'html.parser')
    
    #************ HTML LOCAL *******************
    #page = open('teste-2008-2013.html')
    #content = BeautifulSoup(page, 'html.parser')
    #page.close()
    #************ HTML LOCAL *******************
    
    # Capturando dados dos indicadores, por ano 'a' do intervalo
    registro = []
    for a in range(3, 6):

        #cidade = 'São Paulo' 
        #uf = 'SP'
        ano = intervalo[a]
        if debug==True:
            print('Ano: ', ano)
            
        # Novo registro Municipio / Ano
        data = {}
        data["MUNICIPIO"] = municipio
        data["UF"] = uf
        data["ANO"] = ano

        # Indicadores (tabelas)
        indicadores = content.find_all('table', class_='table')
        for i in range(0, len(indicadores)):

            #print('Indicador: ', i+1)        

            # Subindicador 'j_i', do indicador 'i'
            subindicador_i = indicadores[i].findAll(True, {"class":["rowA", "rowB"]})
            for j in range(0, len(subindicador_i)):

                c = subindicador_i[j].find('div', class_='codigo')
                c = c.get_text().replace('\n', '').replace('\t', '').replace('\t', '').replace('\r', '').replace('\xa0', '')
                
                # Dados do subindicador 'j_i' para o ano 'a'
                valores = subindicador_i[j].find_all('div', class_='number')
                v = valores[a].get_text()
                # Tratamentos
                v = v.replace('\n', '').replace('\t', '').replace('\t', '').replace('\r', '').replace('\xa0', '') # Chars
                v = v.replace('R$', '').replace('%', '') # Symbols
                v = v.replace('.', '').replace(',', '.') # Decimal format

                #print('Subindicador ', i+1, '.', j+1, ': ', v)
                data[c] = v

        #registro = [municipio, uf, str(ano)] + registro
        registro.append(data)
        
    # Arquivando registro  
    dados = pd.DataFrame(registro)
    
    return dados

## 4. Realizando a coleta de dados
---
Para cada município da base de dados SIOPE é realizada a coleta de informações dos indicadores sócio-econômicos dentro do período disponível na plataforma. Por fim, um conjunto de dados único é estabelecido, contemplando para cada município-ano os indicadores disponíveis no portal.

In [ ]:
# Coletando registro historico (SIOPE) de cada municipio
dados = pd.DataFrame(columns=campos)

i = 1
for index, row in municipios.iterrows():

    # 2008 -> 2013
    historico_1 = dados2008_2013( row["Codigo"], row["Nome"], row["CodUF"], row["UF"], campos )
    dados = dados.append(historico_1, ignore_index=True)

    # 2014 -> 2016
    historico_2 = dados2014_2016( row["Codigo"], row["Nome"], row["CodUF"], row["UF"], campos )
    dados = dados.append(historico_2, ignore_index=True)
    
    print(str(i)+'/'+str(municipios.shape[0]))
    i = i + 1

# Criando CSV 
salvar = input('Deseja salvar os dados? (S)im ou (N)ão: ')
if salvar.lower() == 's':
    dados.to_csv('dados_siope/indicadores.csv', sep=';', encoding='utf-8', index=False)